## Setup

In [ ]:
import sys
from pathlib import Path

from aocd import get_data, submit

In [2]:
# Add parent directory to path to allow relative imports into Jupyter notebook
sys.path.append(str(Path.cwd().parent))

In [109]:
# Get raw advent-of-code data
data: str = get_data(year=2024, day=12)

## Part a

In [ ]:
# Imports
from common.utils.dict_grid import print_grid, text_to_dict


In [178]:
def find_regions_in_garden(garden: dict[complex, str]) -> list[tuple[str, set[complex], set[tuple[complex, complex]]]]:
    """Find regions of plots with the same plant and their fences within a garden."""
    visited = set()
    regions = []

    def find_region_from_plot(
        plot: complex,
        plant: str,
    ) -> tuple[set[complex], set[tuple[complex, complex]]]:
        # Skip if plot is not part of the garden, has a different plant, or is already visited
        if plot not in garden or garden[plot] != plant or plot in visited:
            return set(), set()

        # Initialize region and fences
        region, fences = {plot}, set()

        # Add plot to visited
        visited.add(plot)

        # Check all 4 direct neighbors
        for direction in [1, -1, 1j, -1j]:
            next_plot = plot + direction

            # Add fence if next plot is not part of the garden or has a different plant
            if next_plot not in garden or garden[next_plot] != plant:
                # Direction of fence in the fence plot is opposite of the direction from which the plot was checked
                fences.add((plot, -direction))
            else:
                # Recursively check the next plot
                sub_region, sub_fences = find_region_from_plot(next_plot, plant)

                # Add sub region and fences to the current region and fences
                region |= sub_region
                fences |= sub_fences

        return region, fences

    # Iterate over all plots in the garden
    for plot, plant in garden.items():
        if plot not in visited:
            # Find the region and set of fences for each plot that is not part of a visited region
            region, fences = find_region_from_plot(plot, plant)
            regions.append((plant, region, fences))

    return regions

In [ ]:
# Parse data into grid
grid = text_to_dict(data)
print_grid(grid)

In [179]:
# Find regions in grid
regions = find_regions_in_garden(grid)

# Find total fence cost (area of each region multiplied by its number of fences)
total_fence_cost_part_a = sum(len(region) * len(fences) for _, region, fences in regions)

In [ ]:
# Submit answer
submit(total_fence_cost_part_a, part="a", day=12, year=2024)

## Part b

In [ ]:
# Functions
def find_sides(fences: set[tuple[complex, complex]]) -> set[tuple[bool, set[tuple[complex, complex]]]]:
    """Find the sides (straight sections) of a collection of fences."""
    visited = set()
    sides = set()

    def check_fence(fence: tuple[complex, complex], section: tuple[bool, set[tuple[complex, complex]]]) -> None:
        """Check a fence plot to determine if it should be added to a section. Recursively checks neighboring plots."""
        if fence in visited or fence not in fences:
            return

        # Unpack section tuple
        section_is_vertical, section_fences = section

        # Add fence to visited and section
        visited.add(fence)
        section_fences.add(fence)

        # Unpack fence tuple
        fence_plot, fence_side_direction = fence

        # Check the neighbors in the section direction
        for next_plot in [fence_plot + d for d in ([1j, -1j] if section_is_vertical else [1, -1])]:
            # Find the next fence plot in the same direction, if it exists
            if next_fence := (next_plot, fence_side_direction) if (next_plot, fence_side_direction) in fences else None:
                check_fence(next_fence, section)

    # Iterate over all fences
    for fence in fences:
        if fence not in visited:
            # If the fence is on the left or right side of the plot, its side-direction has a real part,
            # and the section is vertical.
            fence_side_direction = fence[1]
            section_is_vertical = bool(fence_side_direction.real)

            # Start a new section for each fence plot that is not part of a visited side
            section_fences = set()

            # Check the fence plot and all connected plots
            check_fence(fence, (section_is_vertical, section_fences))

            # Store the side (collection of fence plots) and its direction
            sides.add((section_is_vertical, frozenset(section_fences)))

    return sides

In [182]:
# Find total fence cost (area of each region multiplied by its number of fence sides)
total_fence_cost_part_b = sum(len(region) * len(find_sides(fences)) for _, region, fences in regions)

In [ ]:
# Submit answer
submit(total_fence_cost_part_b, part="b", day=12, year=2024)